# Filtro de las imagenes

El objetivo de este notebook es mostrar el proceso sobre cómo se filtraron las imagenes disponibles en el dataset según el porcentaje de cobertura de nubes. Para ello se listaron todas las imagenes descargadas, cada imagen se representa como una carpeta. Posteriormente, se cargó el raster con la máscara de nubes se calculó la media de toda la máscara para cada imagen. El resultado se almacenó en un archivo **JSON** con el nombre de *cloud_mask_report.json*, este archivo se emplea en otros notebook para seleccionar las imagenes multiespectrales que cuenten con un porcentaje de nubes inferior a un valor dado

## Librerias

A continuación vamos a cargar algunas librerias para realizar este proceso

In [1]:
from sklearn.preprocessing import MinMaxScaler
from typing import List
from matplotlib import colors
from skimage import exposure
from tqdm import tqdm

import rasterio as rio
import rasterio.plot as rio_plot
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import json

# Semilla pseudoaleatoria
np.random.seed(24)

# Tamaño de las figuras
plt.rcParams["figure.figsize"] = (20,10)

# Ruta base de la carpeta de datos
DATA_BASE = "/home/ggonzr_cloud/deeplearn/data"

### Carga de imágenes

A continuación vamos a buscar las rutas de acceso a las imagenes.

In [2]:
# Directorios de las etiquetas y la imagenes
LABEL_DIR = f"{DATA_BASE}/ref_landcovernet_v1_labels"
IMAGE_DIR = f"{DATA_BASE}/ref_landcovernet_v1_source"

In [3]:
# Listar los subdirectorios de cada carpeta: Etiquetas
label_dirs = glob.glob(f"{LABEL_DIR}/*")
print(f"Número de etiquetas disponibles: {len(label_dirs)}")

Número de etiquetas disponibles: 1982


In [4]:
# Listar los subdirectorios de cada carpeta: Etiquetas
image_dirs = glob.glob(f"{IMAGE_DIR}/*")
print(f"Número de etiquetas disponibles: {len(image_dirs)}")

Número de etiquetas disponibles: 82430


En las siguientes dos celdas podrá observar un ejemplo de la ruta de acceso tanto de las imagenes satelitales como de sus etiquetas. Cada una de ellas se representa en una carpeta

In [5]:
label_dirs[0]

'/home/ggonzr_cloud/deeplearn/data/ref_landcovernet_v1_labels/ref_landcovernet_v1_labels_36PZC_07'

In [6]:
image_dirs[0]

'/home/ggonzr_cloud/deeplearn/data/ref_landcovernet_v1_source/ref_landcovernet_v1_source_33LYE_11_20180818'

A continuación vamos a calcular la media de la máscara de nubes de cada imagen

## Conteo de máscaras

Para el cálculo de la media de la máscara de nubes recibimos como entrada la ruta de la carpeta de la imagen.
Posteriormente, obtenemos la ruta del archivo de la máscara de nubes (CLD.tif) y con ello realizamos la carga del raster empleando la libreria **rasterio**. Para culminar calculamos la media con la funcion **np.mean** y retornamos un diccionario con el ID de la imagen, el ID de la toma de la imagen (chip_id), la ruta de esta imagen en el sistema de archivos de la máquina de trabajo y la media de la máscara de nubes

In [12]:
# El siguiente diccionario va a almacenar la media de la máscara de nubes
def check_mask(path_img: str) -> dict:
    # ID de la imagen
    image_name = path_img.split("/")[-1]
    mask_split = image_name.split("_")
    image_id = f"{mask_split[-3]}_{mask_split[-2]}_{mask_split[-1]}"
    chip_id = f"{mask_split[-3]}_{mask_split[-2]}"
    
    # Media de conteo
    cloud_mask = None
    raster_path = glob.glob(f"{path_img}/CLD.*")[0]
    with rio.open(raster_path) as rf:
        cloud_mask = rf.read(1)
        
    cloud_mask_mean = np.mean(cloud_mask)
    
    # Dar retorno
    return {
        "image_id": image_id,
        "chip_id": chip_id,
        "image_folder_path": path_img,
        "cloud_mask_mean": cloud_mask_mean
    }

In [13]:
# Lista con los datos
images_cloud_mask = []

Con la definición de la función, ejecutamos el proceso. Al final ocurre un error con el procesamiento debido a que la carpeta contenia otros archivos relacionados con metadatos que no seguian el patrón de nombramiento.

In [14]:
for image_path in tqdm(image_dirs):
    mask_data = check_mask(path_img=image_path)
    images_cloud_mask.append(mask_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 82398/82430 [30:46<00:00, 44.62it/s]


IndexError: list index out of range

Para culminar, calculamos el tamaño del arreglo con las imágenes procesadas

In [15]:
len(images_cloud_mask)

82398

Finalmente, guardamos el archivo **JSON** para emplear estos datos en otros notebooks

In [16]:
# Guardar datos
with open(f"{DATA_BASE}/cloud_mask_report.json", "w", encoding="utf-8") as v_f:
    json.dump(images_cloud_mask, v_f, ensure_ascii=False, indent=4)